## Data Loading, Inspection and Visualization
Mainly using pandas


In [ ]:
# Render our plots inline
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
# Import requiered packages
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

We type some code to simply change the visual style of the plots. (The code below is optional and not necessary, and for now you do not need to understand what exactly is happening.)

In [ ]:
# Modifying the style of the graphs
matplotlib.style.use(['seaborn-talk', 'seaborn-ticks', 'seaborn-whitegrid'])

In [ ]:
# Supress warnings
import warnings
warnings.filterwarnings('ignore')

### Data Loading
One can directly load a csv file from an url with pandas or download it locally first and upload it from the local directory on the computer. The `pd.read_csv` method has many options, and you can further read in the [online documentation](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.parsers.read_csv.html). 
In the following, we will focus on the [NYPD Vehicle Collisions](https://data.cityofnewyork.us/Public-Safety/NYPD-Motor-Vehicle-Collisions/h9gi-nx95/data) data set.

In [ ]:
url = 'https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv?accessType=DOWNLOAD'
df = pd.read_csv(url, low_memory=False)

# Let's take a look at the first 5 rows of the dataframe
df.head(5)

### Data Inspection/Visualization
Using the `info()` method you can obtain a concise summary of the data, including the data types under which each column has been saved.

We can use the method `describe()` to get some statistics of the numeric attributes in the DataFrame.

In [ ]:
df.info()

In [ ]:
df.describe()

The `shape` property allows you to see how many rows and columns there are. 

In [ ]:
df.shape

In [ ]:
# Number of rows/observations - the data numerosity
df.shape[0]

In [ ]:
# Number of features/column/attributes - the data dimensionality
df.shape[1]

We can also list the columns and check the data types for each column using `dtypes`.

In [ ]:
df.columns

In [ ]:
df.dtypes

The `object` type is a string. For some of them, we would like to change the data types using for example the `pd.to_datetime` functions. To this end, we first need to understand how to [parse dates using the Python conventions](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior).

The relevant entries from the table are:
* `%m` Month as a zero-padded decimal number.
* `%d` 	Day of the month as a zero-padded decimal number.
* `%Y` Year with century as a decimal number.

Now, we can specify how to parse the dates.

In [ ]:
df["CRASH DATE"] = pd.to_datetime(df["CRASH DATE"], format="%m/%d/%Y")

#### Selecting columns

In [ ]:
cols = ["CRASH DATE", "BOROUGH", "NUMBER OF PERSONS INJURED"]
df.loc[:,cols]
# df[["CRASH DATE", "BOROUGH", "NUMBER OF PERSONS INJURED"]]

#### Selecting rows

In [ ]:
df[0:5]

#### Selecting both rows and columns by name (`df.loc`) or by position (`df.iloc`)

In [ ]:
df.loc[0:5, ["CRASH DATE", "BOROUGH", "NUMBER OF PERSONS INJURED"]]

In [ ]:
df.iloc[[1,4], 0:3]

In [ ]:
# You can also change the value of an observation directly in the data frame
# df.loc[0, "BOROUGH"] = "BROOKLYN"

#### Boolean Indexing
To filter rows of a certain kind. Below an example to select data in certain area specified by latitude and longitude ranges.

In [ ]:
boolean_condition = (df.LONGITUDE<-50) & (df.LONGITUDE>-74.5) & (df.LATITUDE< 41)
df_filtered = df[boolean_condition]
# same as:
# df_filtered = df[(df.LONGITUDE<-50) & (df.LONGITUDE>-74.5) & (df.LATITUDE< 41)]

#### Aggregation
Below you find some aggregation examples.

In [ ]:
# Sum up the number of all injured persons per borough for all different boroughs
df.groupby("BOROUGH", as_index=False)["NUMBER OF PERSONS INJURED"].sum()

In [ ]:
# Apply multiple aggregation functions (here "sum" and "max")
df.groupby("BOROUGH", as_index=False)["NUMBER OF PERSONS INJURED"].agg({"SUM INJURED": "sum", "MAX INJURED": "max"})

In [ ]:
# Apply an aggregating function on multiple variables by multiple aggregating dimensions
df.groupby(["BOROUGH","VEHICLE TYPE CODE 1"])[["NUMBER OF PERSONS INJURED","CONTRIBUTING FACTOR VEHICLE 1"]].sum()

#### Histograms
One can examine the distribution of values by using the `hist` command of Pandas, which creates a histogram. (The histogram is also available as `plot.hist()`, or `plot(kind='hist')`).

In [ ]:
df["NUMBER OF PERSONS INJURED"].hist()
# df_filtered["NUMBER OF PERSONS INJURED"].plot(kind='hist')

By default, the histogram has ~10 bars. We can change the resolution of the histogram using the `bins` attribute. Larger numbers of `bins` allow for higher resolution. 

In [ ]:
df["NUMBER OF PERSONS INJURED"].hist(bins=50)

In [ ]:
# A quick exposure to various options of the "hist" command
df["NUMBER OF PERSONS INJURED"].hist(
    bins=20, # use 20 bars
    range=(0,10), # x-axis from 0 to 10
    density=False,  # show normalized count (density=True), or raw counts (density= False)
    figsize=(15,5), # controls the size of the plot
    alpha=0.8, # make the plot 20% transparent
    color='green' # change color
)

#### Kernel Density Estimation (KDE)

An alternative to histograms is to use the **kernel density**, which estimates a continuous function, instead of the bucketized counts, which tends to be discontinuous and bumpy. We can access this usind the `.plot(kind='kde')` command. 
Let's see an example.

In [ ]:
df["NUMBER OF PERSONS INJURED"].plot(
    kind='kde', 
    color='Black', 
    xlim=(0,5), 
    figsize=(15,5)
)

#### Analyzing the content of categorical columns

We can also get quick statistics about the common values that appear in each column:

In [ ]:
df["BOROUGH"].value_counts()

And we can use the "plot" command to plot the resulting histogram (more detail at http://pandas.pydata.org/pandas-docs/stable/visualization.html).

In [ ]:
df["BOROUGH"].value_counts().plot(kind='bar')

In [ ]:
# horizontal bars (another way to access column in Pandas when there aren't empty spaces in the column name)
df_filtered.BOROUGH.value_counts().plot(kind='barh')

#### Pivot Tables
[Pivot tables](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.pivot_table.html) is one of the most commonly used exploratory tools, and in Pandas they are extremely flexible.

Let's use them to break down the accidents by borough and contributing factor.

In [ ]:
pivot = pd.pivot_table(
    data = df,
    index = 'CONTRIBUTING FACTOR VEHICLE 1',
    columns = 'BOROUGH',
    aggfunc = 'count',
    values = 'COLLISION_ID'
)
pivot.head(10)

### Examples

**Example 1:** Find the dates with most accidents.

In [ ]:
df["CRASH DATE"].value_counts().head(10)

**Example 2:** Find out the 10 most common contributing factors to the collisions. 
 

In [ ]:
df_filtered['CONTRIBUTING FACTOR VEHICLE 1'].value_counts().head(11)

Now let's plot a histogram of the above list. Note that we skip the first element.

In [ ]:
df_filtered['CONTRIBUTING FACTOR VEHICLE 1'].value_counts()[1:11].plot(kind='barh')

**Example 3:** Find out how many collisions had 0 person injured, 1 person injured, etc. persons injured in each accident. 

The `.plot(logy=True)` option is used in the plot to make the y-axis logarigthmic.
 

In [ ]:
plot = (
    df['NUMBER OF PERSONS INJURED'] # take the num of injuries column
    .value_counts() # compure the freuquency of each value
    .sort_index() # sort the results based on the index value instead of the frequency, 
                  # which is the default for value_counts
    .plot( # and plot the results
        kind='line', # we use a line plot because the x-axis is numeric/continuous
        marker='o',  # we use a marker to mark where we have data points 
        logy=True # make the y-axis logarithmic
    )
)
plot.set_xlabel("Number of injuries")
plot.set_ylabel("Number of collisions");

**Example 3:** Plot the number of accidents per day. 
Ensure that your date column is in the right datatype and that it is properly sorted, before plotting. The [resample](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.resample.html) command is used to change the frequency from one day, to, say, a month. The `drop` command is used to delete rows or columns.


In [ ]:
# Date converted to proper date format
df["CRASH DATE"] = pd.to_datetime(df["CRASH DATE"], format="%m/%d/%Y")

In [ ]:
(
    df["CRASH DATE"].value_counts() # count the number of accidents per day
    .sort_index() # sort the dates
    .resample('1M') # take periods of 1 month
    .sum() # sum the number of accidents per month
    .drop(pd.to_datetime('2019-04-30'), axis='index') # drop the current month
    .plot() # plot the result
)

**Example 4:** Plot the accidents in map. To do this we use a scatter plot using the `plot(kind='scatter', x=..., y=....)` command, and use the `LATITUDE` and `LONGITUDE` parameters.

In [ ]:
# We do data filtering by specifying a selection condition to limit the lat/long values 
# to be values idicating the NYC region. Remaining are probably wrong inputs.
cleandf = df[(df.LONGITUDE<-50) & (df.LONGITUDE>-74.5) & (df.LATITUDE< 41)]

cleandf[ (df.LATITUDE>40) & (df.LATITUDE<41) & (df.LONGITUDE> -74.6) & (df.LONGITUDE<-50) ].plot(
    figsize = (20,15),
    kind = 'scatter',
    x = 'LONGITUDE',
    y = 'LATITUDE',
    s = 1, # make each dot to be very small 
    alpha = 0.05 # makes each point 95% transparent
)


### 2d histograms, density plots, and contour plots
In the picture above, we can visually see that Manhattan, especially eastern midtown, and the area downtown near the entrance to the bridges, has a higher density. We can also derive histograms and density plots on 2-dimensions.

#### Hexagonal bin plot
The hexbin plot created a 2-d histogram, where the color signals the number of points within a particular area. The `gridsize` parameter indicates the number of hexagones in the x direction. Higher values offer higher granularity, but very high values tend to create sparsity, when we do not have enough data points.

In [ ]:
# Hexbin plot
cleandf.plot(
    kind='hexbin',
    x='LONGITUDE',
    y='LATITUDE',
    gridsize=100,
    cmap=plt.cm.Blues,
    figsize=(10, 7))

#### 2d density  and contour plots
An alternative to the hexbin plots is to use density plots in two dimensions.

In [ ]:
# Basic 2D density plot
plt.subplots(figsize=(20, 15))

# We take a sample, because density plots take a long time to compute
# and a sample is typically as good as the full dataset
sample = cleandf.sample(10000)

sns.kdeplot(
    sample.LONGITUDE,
    sample.LATITUDE,
    gridsize=100,  # controls the resolution
    cmap=plt.cm.rainbow,  # color scheme
    shade= True, # whether to have a density plot (True), or just the contours (False)
    alpha=0.5,
    shade_lowest=False,
    n_levels=50  # How many contours/levels to have
)

In [ ]:
# Basic 2D contour plot
plt.subplots(figsize=(20, 15))

# We take a sample, because density plots take a long time to compute
# and a sample is typically as good as the full dataset
sample = cleandf.sample(10000)

sns.kdeplot(
    sample.LONGITUDE,
    sample.LATITUDE,
    gridsize=100,
    cmap=plt.cm.rainbow,
    shade=False,
    shade_lowest=False,
    n_levels=25)

#### Combining Plots
So far, we examined how to create individual plots. We can even combine multiple plots together, using the ax parameter. So, let's say that we want to combine the scatter plots with the contour plot above:

In [ ]:
sample = cleandf.sample(10000)

scatterplot = cleandf.plot(
    kind='scatter',
    x='LONGITUDE',
    y='LATITUDE',
    figsize=(20, 15),
    s=0.5,
    alpha=0.1)

sns.kdeplot(
    sample.LONGITUDE,
    sample.LATITUDE,
    gridsize=100,
    cmap=plt.cm.rainbow,
    shade=False,
    shade_lowest=False,
    n_levels=20,
    alpha=1,
    ax=scatterplot)